<a href="https://colab.research.google.com/github/C8XY66/GNN/blob/main/GIN_ModelCheckpoint_EvFr_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install git+https://github.com/optuna/optuna.git
!pip install optuna-dashboard

In [4]:
import os
import re
import csv
import datetime
import pytz
import numpy as np
from typing import Optional
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn.functional as F
from torchmetrics import Accuracy

import torch_geometric.transforms as T
from torch_geometric.data.lightning import LightningDataset
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GIN, MLP, global_add_pool
from torch_geometric.data import InMemoryDataset

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
# Load the TensorBoard notebook extension
%load_ext tensorboard

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization.matplotlib import plot_contour, plot_edf, plot_intermediate_values, plot_optimization_history, plot_parallel_coordinate, plot_param_importances, plot_slice

from google.colab import drive
drive.mount('/content/gdrive')
MAIN_DIR = "/content/gdrive/My Drive/ColabNotebooks/" 



import logging

logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.callbacks.early_stopping").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.callbacks.model_checkpoint").setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pytorch_lightning.callbacks.model_checkpoint")
warnings.filterwarnings("ignore", category=UserWarning, module="pytorch_lightning.utilities")




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# Check for CUDA system support and use GPU if available otherwise run on CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # device = Context-manager that changes the selected device
print(device)

cuda


In [44]:
# Hyperparameters

DATASET_NAME = 'NCI1'
lr = 0.01 
EPOCHS = 20 #final = 1000
SEED = 42
N_SPLITS = 5
REP = 2

# Reloading interrupted run
PARENT_DIR = None

#RUN = "NCI1_reps_2_folds_5_epochs_100_2023-04-26_09-00"
#PARENT_DIR = os.path.join(MAIN_DIR, "logs", RUN)

STARTING_REP = 0
STARTING_FOLD = 0


In [7]:
class GNNModel(pl.LightningModule):
    def __init__(self, in_channels: int, out_channels: int,
                 hidden_channels: int, dropout, num_layers=5):
        super().__init__()
        self.save_hyperparameters()
        self.gnn = GIN(in_channels, hidden_channels, num_layers,
                       dropout=dropout, jk='cat')

        self.classifier = MLP([hidden_channels, hidden_channels, out_channels],
                              norm="batch_norm", dropout=dropout)

        self.train_acc = Accuracy(task='multiclass', num_classes=out_channels)
        self.val_acc = Accuracy(task='multiclass', num_classes=out_channels)
        self.test_acc = Accuracy(task='multiclass', num_classes=out_channels)

    def forward(self, x, edge_index, batch):
        x = self.gnn(x, edge_index)
        x = global_add_pool(x, batch)
        x = self.classifier(x)
        return x

    def training_step(self, data, batch_idx):
        y_hat = self(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(y_hat, data.y)
        self.train_acc(y_hat.softmax(dim=-1), data.y)
        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_acc', self.train_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, data, batch_idx):
        y_hat = self(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(y_hat, data.y)
        self.val_acc(y_hat.softmax(dim=-1), data.y)
        self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_acc', self.val_acc, prog_bar=True, on_step=False, on_epoch=True)

    def test_step(self, data, batch_idx):
        y_hat = self(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(y_hat, data.y)
        self.test_acc(y_hat.softmax(dim=-1), data.y)
        self.log('test_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('test_acc', self.test_acc, prog_bar=True, on_step=False,on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

    def on_save_checkpoint(self, checkpoint):
        checkpoint["init_args"] = self.hparams

In [8]:
class GraphDataModule(pl.LightningDataModule):
    def __init__(self, dataset_name, n_splits=10, fold=0, seed=None, num_workers=0):
        super().__init__()
        self.dataset_name = dataset_name
        self.n_splits = n_splits
        self.fold = fold
        self.seed = seed
        self.num_workers = num_workers

    def prepare_data(self):
        if self.seed is not None:
            torch.manual_seed(self.seed)
            np.random.seed(self.seed)    
        
        self.dataset = TUDataset(root='data/TUDataset', name=self.dataset_name)
        #self.dataset = self.dataset[:1000] #for quick experiments
        self.skf = StratifiedKFold(n_splits=self.n_splits)

    def setup(self, stage: Optional[str] = None, fold: int = 0, batch_size: int = 32):
        self.fold = fold
        self.batch_size = batch_size
        y = [data.y.item() for data in self.dataset]

        train_indices, test_indices = list(self.skf.split(torch.zeros(len(y)), y))[self.fold]
        train_dataset = self.dataset[train_indices]
        
        num_val = int(len(train_dataset) * 0.1)
        num_train = len(train_dataset) - num_val
        
        self.train_dataset, self.val_dataset = torch.utils.data.random_split(train_dataset, [num_train, num_val])
        self.test_dataset = self.dataset[test_indices]
      
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    @property
    def num_node_features(self):
        return self.dataset.num_node_features

    @property
    def num_classes(self):
        return self.dataset.num_classes


In [43]:
def create_log_dir(repetition_index, fold_index):
    global PARENT_DIR
    
    # Current timestamp
    now = datetime.datetime.now(pytz.timezone('Europe/Zurich')).strftime("%Y-%m-%d_%H-%M")

    # Parent directory
    parent_dir_info = f"{DATASET_NAME}_reps_{REP}_folds_{N_SPLITS}_epochs_{EPOCHS}"

    if PARENT_DIR is None:
        PARENT_DIR = f"{MAIN_DIR}logs/{parent_dir_info}_{now}"
        if not os.path.exists(PARENT_DIR):
            os.makedirs(PARENT_DIR)


     # Subdirectory for the specific repetition and fold
    if repetition_index is not None and fold_index is not None:
        sub_dir = f"{PARENT_DIR}/rep_{repetition_index}_fold_{fold_index}"
        if not os.path.exists(sub_dir):
            os.makedirs(sub_dir)
    else:
        sub_dir = PARENT_DIR
    
    return sub_dir

In [10]:
def create_trainer(log_dir, epochs, pruning_callback=None, testing=False, trial=None):
    
    callbacks = []
    
    if not testing:

        # Training Callbacks
        early_stopping = EarlyStopping(monitor="val_acc", mode="max", patience=10, verbose=True)
        callbacks.append(early_stopping)
        
        model_checkpoint = ModelCheckpoint(dirpath=os.path.join(log_dir, "checkpoints"), 
                                           filename=f"model_trial_{trial.number}",
                                           save_top_k=1, 
                                           monitor="val_acc", 
                                           mode="max", 
                                           auto_insert_metric_name=True,
                                           verbose=True)
        callbacks.append(model_checkpoint)

        if pruning_callback is not None:
            callbacks.append(pruning_callback)

    # Create trainer
    trainer = pl.Trainer(
        callbacks=callbacks,
        max_epochs=epochs,
        log_every_n_steps=5,
        logger=TensorBoardLogger(save_dir=log_dir),
        enable_progress_bar=False,
        enable_model_summary=False,
    )

    return trainer

In [41]:
def objective(trial, datamodule, epochs, repetition_index, fold_index): 

    # Optimise hyperparameters
    hidden_channels = trial.suggest_categorical('hidden_channels', [16, 32])
    batch_size = trial.suggest_categorical('batch_size', [32, 128])
    dropout = trial.suggest_categorical('dropout', [0.0, 0.5])

    # Model and DataModule
    datamodule.setup(fold=0, batch_size=batch_size)
    model = GNNModel(in_channels=datamodule.num_node_features, 
                     out_channels=datamodule.num_classes, 
                     hidden_channels=hidden_channels, 
                     dropout=dropout)

    # Training
    pruning_callback = PyTorchLightningPruningCallback(trial, monitor="val_acc") #from optuna-pl-integration
    
    log_dir = create_log_dir(repetition_index, fold_index)
    trainer = create_trainer(log_dir, epochs=epochs, 
                             pruning_callback=pruning_callback,
                             trial=trial)
    
    hyperparameters = dict(hidden_channels=hidden_channels, batch_size=batch_size, epochs=epochs, dropout=dropout)
    trainer.logger.log_hyperparams(hyperparameters)    
    
    trainer.fit(model, datamodule=datamodule)

    return trainer.callback_metrics['val_acc'].item()
    

In [12]:
def save_test_results(log_dir, repetition_index, fold_index, test_acc, avg_performance=None, overall_avg_performance=None):
    
    # Create a CSV file to log the test accuracies
    file_path = os.path.join(log_dir, 'test_accuracies.csv')

    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['repetition', 'fold', 'test_accuracy_fold', 'Avg_Performance_Rep', 'Overall_Avg_Performance'])

    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)

        if repetition_index is not None and fold_index is not None:
            row = [repetition_index, fold_index, test_acc]
        else:
            row = ['', '', '']

        if avg_performance is not None:
            row.append(avg_performance)
        else:
            row.append('')

        if overall_avg_performance is not None:
            row.append(overall_avg_performance)
        else:
            row.append('')

        writer.writerow(row)


In [45]:
if __name__ == '__main__':

    overall_performances = [] 

    for r in range(STARTING_REP, REP):
        seed = r + 1  # Set a new seed for each repetition
        datamodule = GraphDataModule(dataset_name=DATASET_NAME, seed=seed)
        datamodule.prepare_data()
        fold_performances = []
        
        for fold in range(STARTING_FOLD if r == STARTING_REP else 0, N_SPLITS):
            log_dir = create_log_dir(r, fold)
            # Create a new study object for each fold
            study = optuna.create_study(direction="maximize", 
                                        pruner=optuna.pruners.MedianPruner(), 
                                        sampler=optuna.samplers.TPESampler(seed=SEED), 
                                        study_name=f"rep_{r}_fold_{fold}",
                                        #storage=f"sqlite:///{log_dir}/rep_{r}_fold_{fold}_optuna.db",
                                        #load_if_exists=True
                                        )
             
            datamodule.setup("fit", fold)
            study.optimize(lambda trial: objective(trial, datamodule, EPOCHS, r, fold), n_trials=8)
            print(f"Best trial for fold {fold}: {study.best_trial.value}")

            # Load the model with the best hyperparameters
            checkpoint_name = f"model_trial_{study.best_trial.number}.ckpt"
            checkpoint_path = os.path.join(log_dir, "checkpoints", checkpoint_name)
            checkpoint = torch.load(checkpoint_path)  # Load the checkpoint dictionary from the file
            init_args = checkpoint["init_args"]  # Access the saved initialization parameters
            best_model = GNNModel(**init_args)  # Initialize the model using the saved parameters
            best_model.load_state_dict(checkpoint['state_dict'])

            # Test the best model
            datamodule.setup("test", fold)
            trainer = create_trainer(log_dir=log_dir, epochs=EPOCHS, testing=True)
            test_result = trainer.test(best_model, datamodule=datamodule)
            test_acc = test_result[0]["test_acc"]
            print(f"Test accuracy for fold {fold}: {test_acc}")

            fold_performances.append(test_acc)

        avg_performance = np.mean(fold_performances)
        print(f"Average performance for repetition {r}: {avg_performance}")
        overall_performances.append(avg_performance)

        # Save test accuracies, average performance, and overall average performance after all folds are done
        for fold, test_acc in enumerate(fold_performances):
            if fold == N_SPLITS - 1:
                save_test_results(PARENT_DIR, r, fold, test_acc, avg_performance)
            else:
                save_test_results(PARENT_DIR, r, fold, test_acc)

    overall_avg_performance = np.mean(overall_performances)
    print(f"Overall average performance: {overall_avg_performance}")
    save_test_results(PARENT_DIR, None, None, None, None, overall_avg_performance)


[I 2023-04-26 09:05:54,304] A new study created in memory with name: rep_0_fold_0
[I 2023-04-26 09:06:25,979] Trial 0 finished with value: 0.6829268336296082 and parameters: {'hidden_channels': 32, 'batch_size': 32, 'dropout': 0.0}. Best is trial 0 with value: 0.6829268336296082.
[I 2023-04-26 09:06:57,249] Trial 1 finished with value: 0.7100270986557007 and parameters: {'hidden_channels': 32, 'batch_size': 128, 'dropout': 0.5}. Best is trial 1 with value: 0.7100270986557007.
[I 2023-04-26 09:07:26,528] Trial 2 finished with value: 0.7046070694923401 and parameters: {'hidden_channels': 16, 'batch_size': 128, 'dropout': 0.5}. Best is trial 1 with value: 0.7100270986557007.
[I 2023-04-26 09:07:51,704] Trial 3 finished with value: 0.6910569071769714 and parameters: {'hidden_channels': 16, 'batch_size': 32, 'dropout': 0.5}. Best is trial 1 with value: 0.7100270986557007.
[I 2023-04-26 09:08:25,146] Trial 4 finished with value: 0.7018970251083374 and parameters: {'hidden_channels': 32, 'bat

Best trial for fold 0: 0.7100270986557007


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6155717968940735     │
│         test_loss         │    0.6637427806854248     │
└───────────────────────────┴───────────────────────────┘

[I 2023-04-26 09:08:32,258] A new study created in memory with name: rep_0_fold_1


Test accuracy for fold 0: 0.6155717968940735


...

Test accuracy for fold 3: 0.6277372241020203


[I 2023-04-26 09:31:48,528] Trial 0 finished with value: 0.7046070694923401 and parameters: {'hidden_channels': 32, 'batch_size': 32, 'dropout': 0.0}. Best is trial 0 with value: 0.7046070694923401.
[I 2023-04-26 09:32:10,041] Trial 1 finished with value: 0.7018970251083374 and parameters: {'hidden_channels': 32, 'batch_size': 128, 'dropout': 0.5}. Best is trial 0 with value: 0.7046070694923401.
[I 2023-04-26 09:32:36,397] Trial 2 finished with value: 0.7154471278190613 and parameters: {'hidden_channels': 16, 'batch_size': 128, 'dropout': 0.5}. Best is trial 2 with value: 0.7154471278190613.
[I 2023-04-26 09:33:07,686] Trial 3 finished with value: 0.696476936340332 and parameters: {'hidden_channels': 16, 'batch_size': 32, 'dropout': 0.5}. Best is trial 2 with value: 0.7154471278190613.
[I 2023-04-26 09:33:36,719] Trial 4 finished with value: 0.6612465977668762 and parameters: {'hidden_channels': 32, 'batch_size': 128, 'dropout': 0.0}. Best is trial 2 with value: 0.7154471278190613.
[I 

Best trial for fold 4: 0.7154471278190613


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6447688341140747     │
│         test_loss         │    0.6352428197860718     │
└───────────────────────────┴───────────────────────────┘

Test accuracy for fold 4: 0.6447688341140747
Average performance for repetition 1: 0.6350365042686462
Overall average performance: 0.6321167945861816


In [ ]:
#%tensorboard --logdir '/content/gdrive/My Drive/ColabNotebooks/logs'
%tensorboard --logdir '{MAIN_DIR}'